# **Introduction:**

This file serves to compare the inference systems that had been developed.

**Date Created: 16/2/2025**

**Date Modified: 18/2/2025**

# **Import Packages:**

This section imports the necessary packages.

In [232]:
# import packages:
import numpy as np
import time
import os
import pandas as pd
from keras.models import load_model
import tensorflow as tf
from tensorflow.keras.losses import MeanSquaredError
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, root_mean_squared_error
from pickle import load
from ANFIS_Custom_Layers import *
from PythonFISFunctionV3 import *

# **Pre-amble:**

This section defines paths and variables to be used in the model loading.

In [233]:
# define the ANN directory path:
ann_path = os.path.join(os.getcwd(), 'ANN_Model')

# define the ANFIS directory path:
anfis_path = os.path.join(os.getcwd(), 'ANFIS_Model')

# define the dictionary of custom objects for the ANFIS:
custom_objects = {
    # # layers:
    'MF_Layer'          : MF_Layer,
    'FS_Layer'          : FS_Layer,
    'NM_Layer'          : NM_Layer,
    'CN_Layer'          : CN_Layer,
    'O_Layer'           : O_Layer,

    # other:
    'OrderedConstraint' : OrderedConstraint(),
    'mse'               : MeanSquaredError()
}

# define the batch size for model warming:
batch_size = 1

# **Create FIS:**

This section creates the rule-base for the FIS.

In [234]:
rulebase = fis_create()

# **Load the ANN Model:**

This section loads the ANN model and the scaler that is used for the ANN.

In [235]:
# load the ann model:
ann_model = load_model(ann_path + '/ann_model.h5', custom_objects = {'mse' : MeanSquaredError()})

# load the ann scaler:
ann_scaler = load(open(ann_path + '/ann_scaler.pkl', 'rb'))

# need to warm the model to prevent retracing of computational graph on first inference:
dummy_input = np.zeros((batch_size, 3), dtype = np.float32)
dummy_input = ann_scaler.transform(dummy_input)
ann_model.predict(dummy_input)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step


c:\Users\mtidd2\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[9.33442]], dtype=float32)

# **Load the ANFIS Model:**

This section loads the ANFIS model and the scaler.

In [236]:
# load the model:
anfis_model = load_model(anfis_path + '/anfis_model.h5', custom_objects = custom_objects)
anfis_model.compile(optimizer="adam", loss="mse") 

# load the scaler:
anfis_scaler = load(open(anfis_path + '/anfis_scaler.pkl', 'rb'))

# warmup the model:
dummy_input = np.zeros((batch_size, 3), dtype=np.float32)  
dummy_input = anfis_scaler.transform(dummy_input)
anfis_model.predict(dummy_input)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


array([[9.028122]], dtype=float32)

# **Generate Testing Data:**

This section generates the data that the models are tested on. This is done by sampling the universe of discourse of each variable and predicting the suitability using the FIS.

In [237]:
# initialize testing data:
test_data = []

# generate data:
for i in range(100):
    # randomly sample each universe of discourse:
    lh = float(np.random.randint(0, 10 + 1))
    dtt = np.random.uniform(0, 25)
    dh = np.random.uniform(0, 50)

    # infer suitability, get time:
    fis_start_time = time.time()
    suit = fis_solve(rulebase, lh, dtt, dh)
    fis_time = time.time() - fis_start_time

    # append to test data:
    test_data.append([lh, dtt, dh, suit, fis_time])

# convert to numpy array:
test_data = np.array(test_data)

Get the total inference time for the FIS:

In [238]:
# get total inference time:
total_fis_time = np.sum(test_data[:, -1])

# get the average inference time:
avg_fis_time = np.average(test_data[:, -1])

# print to user:
print(f'total inference time was: {round(total_fis_time, 3)} seconds')
print(f'average inference time was: {round(avg_fis_time, 3)} seconds')

total inference time was: 9.067 seconds
average inference time was: 0.091 seconds


# **Test the Models:**

This section tests the models against one another on the same input dataset.

Test the ANN:

In [239]:
# scale the inputs:
scaled_inputs = ann_scaler.transform(test_data[:, :3])

# results:
ann_results = []

# do inference:
for a, b, c in scaled_inputs:
    # formulate input:
    input = np.array([[a, b, c]])

    # get start time:
    ann_start_time = time.time()
    ann_prediction = ann_model.predict(input, verbose = 0)
    ann_time = time.time() - ann_start_time

    # append to results:
    ann_results.append([*ann_prediction.flatten(), ann_time])

# convert to numpy array:
ann_results = np.array(ann_results)

c:\Users\mtidd2\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Get the total inference time for the ANN:

In [240]:
# get total inference time:
total_ann_time = np.sum(ann_results[:, -1])

# get the average inference time:
avg_ann_time = np.average(ann_results[:, -1])

# print to user:
print(f'total inference time was: {round(total_ann_time, 3)} seconds')
print(f'average inference time was: {round(avg_ann_time, 3)} seconds')

total inference time was: 3.996 seconds
average inference time was: 0.04 seconds


Test the ANFIS:

In [241]:
# scale the inputs:
scaled_inputs = anfis_scaler.transform(test_data[:, :3])

# results:
anfis_results = []

# do inference:
for a, b, c in scaled_inputs:
    # formulate input:
    input = np.array([[a, b, c]])

    # get start time:
    anfis_start_time = time.time()
    anfis_prediction = anfis_model.predict(input, verbose = 0)
    anfis_time = time.time() - anfis_start_time

    # append to results:
    anfis_results.append([*anfis_prediction.flatten(), anfis_time])

# convert to numpy array:
anfis_results = np.array(anfis_results)

Get the total inference time for the ANFIS:

In [242]:
# get total inference time:
total_anfis_time = np.sum(anfis_results[:, -1])

# get the average inference time:
avg_anfis_time = np.average(anfis_results[:, -1])

# print to user:
print(f'total inference time was: {round(total_anfis_time, 3)} seconds')
print(f'average inference time was: {round(avg_anfis_time, 3)} seconds')

total inference time was: 4.041 seconds
average inference time was: 0.04 seconds


# **Analyze the Results:**

This section analyzes the results from this analysis. It compares the total inference time, the average inference time, the MAE, MSE, RMSE, and the $R^{2}$.

In [243]:
# extract the suitability of each model:
fis_suits = test_data[:, -2]
ann_suits = ann_results[:, 0]
anfis_suits = anfis_results[:, 0]

# calculate the difference between the values:
percentage_diff_ann = (ann_suits - fis_suits) / fis_suits * 100
percentage_diff_anfis = (anfis_suits - fis_suits) / fis_suits * 100

# mean of the absolute percentage differences:
avg_percent_diff_ann = np.mean(np.abs((ann_suits - fis_suits) / fis_suits * 100))
avg_percent_diff_anfis = np.mean(np.abs((anfis_suits - fis_suits) / fis_suits * 100))

# metrics for ann:
ann_mae = mean_absolute_error(fis_suits, ann_suits)
ann_mse = mean_squared_error(fis_suits, ann_suits)
ann_rmse = root_mean_squared_error(fis_suits, ann_suits)
ann_r2 = r2_score(fis_suits, ann_suits)

# amalgamate:
ann_metrics = [ann_mae, ann_mse, ann_rmse, ann_r2, total_ann_time, avg_ann_time]

# metrics for anfis:
anfis_mae = mean_absolute_error(fis_suits, anfis_suits)
anfis_mse = mean_squared_error(fis_suits, anfis_suits)
anfis_rmse = root_mean_squared_error(fis_suits, anfis_suits)
anfis_r2 = r2_score(fis_suits, anfis_suits)

# amalgamate:
anfis_metrics = [anfis_mae, anfis_mse, anfis_rmse, anfis_r2, total_anfis_time, avg_anfis_time]

Generate DataFrames:

In [244]:
# define the data for comparing these suitabilities:
data1 = {'FIS Suitability' : fis_suits, 
        'ANN Suitability' : ann_suits, 
        'ANN-FIS % Difference' : percentage_diff_ann, 
        'ANFIS Suitability' : anfis_suits, 
        'ANFIS-FIS % Difference' : percentage_diff_anfis}

# define the dataframe for suitability differences:
df1 = pd.DataFrame(data1)

# define the data for comparing the metrics:
data2 = {'ANN Metrics' : ann_metrics, 'ANFIS Metrics' : anfis_metrics}

# define the dataframe for metrics:
df2 = pd.DataFrame(data2)
df2.index = ['MAE', 'MSE', 'RMSE', 'R2', 'Total Time (s)', 'Avg. Time (s)']


Display results to user:

In [245]:
# print the metrics:
print(f'metrics are: \n {df2}')

# average percent differences:
print(f'\n mean of the absolute percentage difference for ANN: {round(avg_percent_diff_ann, 3)} %')
print(f'\n mean of the absolute percentage difference for ANFIS: {round(avg_percent_diff_anfis, 3)} %')

metrics are: 
                 ANN Metrics  ANFIS Metrics
MAE                0.076185       0.239555
MSE                0.011472       0.086502
RMSE               0.107108       0.294113
R2                 0.995246       0.964156
Total Time (s)     3.996010       4.040599
Avg. Time (s)      0.039960       0.040406

 mean of the absolute percentage difference for ANN: 2.288 %

 mean of the absolute percentage difference for ANFIS: 7.29 %
